# jieba工具库使用


In [4]:
import jieba

seg_list = jieba.cut("小米在网易云课堂学习自然语言处理", cut_all=True)
print("Full Mode: " + "/".join(seg_list))  # 全模式

seg_list = jieba.cut("小米在网易云课堂学习自然语言处理", cut_all=False)
print("Default Mode: " + "/".join(seg_list))  # 精确模式（默认模式）

seg_list = jieba.cut_for_search("小米在网易云课堂学习自然语言处理")   # 搜索引擎模式
print("，".join(seg_list))  

Full Mode: 小米/在/网易/云/课堂/学习/自然/自然语言/语言/处理
Default Mode: 小米/在/网易/云/课堂/学习/自然语言/处理
小米，在，网易，云，课堂，学习，自然，语言，自然语言，处理


添加用户自定义字典
* 1.jieba.load_userdict(file_name)  
* 2.少量词汇可自己手动添加  
    * add_word(word, freq=None, tag=None)和del_word(word)在程序中动态修改字典  
    * suggest_freq(segment, tune=True)可调节单个词语的词频，使其能（或不能）被分出来

In [5]:
print('/'.join(jieba.cut("如果放到字典中将出错。", HMM=False)))

如果/放到/字典/中将/出错/。


In [6]:
jieba.suggest_freq(('中', '将'), tune=True)
print('/'.join(jieba.cut("如果放到字典中将出错。", HMM=False)))

如果/放到/字典/中/将/出错/。


## 词性标注

In [8]:
import jieba.posseg as pseg

words = pseg.cut("我在网易云学习自然语言处理")
for word, flag in words:
    print("%s  %s" % (word, flag))

我  r
在  p
网易  n
云  ns
学习  v
自然语言  l
处理  v


## 关键词抽取
### 基于TF-IDF算法的关键词抽取
* import jieba.analyse
* jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())

算法细节   
TF(词频)，IDF(逆文档频率)  
**第一步，计算词频** 
 $$词频（TF）= 某个词在文章中出现的次数$$
 考虑到文章有长短之分，为了便于不同文章的比较，进行词频标准化。
 $$词频（TF）= \frac{某个词在文章中出现的次数}{文章的总词数}$$
 或者
  $$词频（TF）= \frac{某个词在文章中出现的次数}{该文出现次数最多的词的出现次数}$$
  
**第二步，计算逆文档频率**
这时，需要一个语料库（corpus），用来模拟语言的使用环境。
$$逆文档频率（IDF）= log(\frac{语料库的文档总数}{包含该词的文档数+1})$$
如果一个词越常见，那么分母就越大，逆文档频率就越接近于0。  

**第三步，计算TF-IDF**
$$TF-IDF=词频(TF)\times逆文档频率(IDF)$$

* jieba.analyse.set_idf_path(file_name)   # 自定义语料库路径
* jieba.analyse.set_stop_words(file_name) # 自定义停用词语料库路径

In [10]:
import jieba.analyse as analyse

lines = open('../Data/NBA.txt', encoding='utf-8').read()
print(" ".join(analyse.extract_tags(lines, topK=20, withWeight=False, allowPOS=())))

韦少 杜兰特 全明星 全明星赛 MVP 威少 正赛 科尔 投篮 勇士 球员 斯布鲁克 更衣柜 NBA 三连庄 张卫平 西部 指导 雷霆 明星队


In [13]:
lines = open('../Data/西游记.txt', encoding='utf-8').read()
print(" ".join(analyse.extract_tags(lines, topK=20, withWeight=False, allowPOS=())))

行者 八戒 师父 三藏 唐僧 大圣 沙僧 妖精 菩萨 和尚 那怪 那里 长老 呆子 徒弟 怎么 不知 老孙 国王 一个


### 基于TextRank算法的关键词抽取
* jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn'))
* jieba.analyse.TextRank()  新建自定义TextRank实例

基本思想：  
* 将待抽取关键词的文本进行分词；
* 以固定的窗口大小(默认为5，通过span属性调整)，词之间的共现关系，构建图；
* 计算图中节点的PageRank，注意是无向带权图

In [14]:
print(" ".join(analyse.textrank(lines, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))))

行者 师父 八戒 三藏 大圣 不知 菩萨 妖精 只见 长老 国王 却说 呆子 徒弟 小妖 出来 不得 不见 不能 师徒
